# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [ ]:
from google.colab import userdata
import os
from openai import OpenAI

# OPEN_API_KEY = userdata.get("OPEN_API_KEY")
# client = OpenAI(api_key=OPEN_API_KEY)

# 환경변수 지정
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client = OpenAI()

## 대화형 챗봇

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 친절한 챗봇입니다.'},
        {'role': 'user', 'content': '안녕, 내 이름은 윤두준이야~'},
        {'role': 'assistant', 'content': '안녕하세요, 윤두준님! 만나서 반갑습니다. 어떻게 도와드릴까요?'},
        {'role': 'user', 'content': '잘 지냈어? 내 이름 기억하니?'}
    ]
)
print(response.choices[0].message.content)

네, 윤두준님! 잘 지냈어요? 항상 여기에서 도와드릴 준비가 되어 있습니다. 궁금한 점이나 이야기하고 싶은 것이 있으면 말씀해 주세요!


In [ ]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': '너는 LLM 전문가입니다.'},
        {'role': 'user', 'content': '안녕, 나는 LLM 꿈나무 윤두준이야~'},
        {'role': 'assistant', 'content': '안녕, 윤두준! LLM에 관심이 많다니 멋지네. 어떤 부분에 대해 배우고 싶거나 궁금한 점이 있니? 함께 이야기해보자!'},
        {'role': 'user', 'content': 'Transformer 모델을 공부하고 싶어.'},
        {'role': 'assistant', 'content': """
        좋아! Transformer 모델은 자연어 처리 분야에서 매우 중요한 개념이야. 기본적으로 Transformer는 2017년에 제안된 모델로, 자기 주의(Self-Attention) 메커니즘을 활용하여 입력 시퀀스의 각 단어 간의 관계를 효과적으로 학습할 수 있어. Transformer의 주요 구성 요소는 다음과 같아:

1. **인코더(Encoder)**: 입력 데이터를 처리하고, 고차원 벡터 표현으로 변환해. 인코더는 여러 개의 레이어로 구성되며, 각각은 자기 주의 메커니즘과 피드포워드 신경망으로 구성돼.

2. **디코더(Decoder)**: 인코더의 출력을 받아 최종 출력을 생성해. 디코더 역시 여러 개의 레이어로 구성되며, 자기 주의 메커니즘 외에도 인코더의 출력을 참조하는 메커니즘이 있어.

3. **자기 주의 메커니즘**: 입력 시퀀스 내의 단어들이 서로 어떻게 관련되어 있는지를 파악해. 각 단어는 다른 모든 단어와의 관계를 고려하여 표현을 업데이트해.

4. **포지셔널 인코딩(Positional Encoding)**: 시퀀스 내에서 각 단어의 위치 정보를 담기 위해 사용해. Transformer는 순서에 대한 정보를 가지지 않기 때문에 이것이 필요해.

Transformer의 이러한 구조는 병렬 처리가 가능하다는 장점이 있고, 자연어 처리 작업에서 성능이 뛰어나기 때문에 BERT, GPT, T5와 같은 다양한 모델들이 Transformer를 기반으로 하고 있어.

더 궁금한 부분이나 구체적인 질문이 있다면 언제든지 물어보세요!
        """},
        {'role': 'user', 'content': '어려워~ 어텐션을 초등학생도 이해할 수 있게 설명해줘.'},
        {'role': 'assistant', 'content': """
        물론이지! 어텐션(Attention) 개념을 초등학생이 이해할 수 있도록 쉽게 풀어볼게.

어텐션은 마치 친구와 대화할 때 듣는 방식과 비슷해. 예를 들어, 학교에서 선생님이 수업을 할 때, 여러 가지 이야기를 해. 그 중에서 우리가 가장 중요하다고 생각하는 부분에 집중하게 돼. 예를 들어, 선생님이 "모든 생물은 세포로 이루어져 있다"라고 말할 때, 우리는 그 부분에 특히 더 귀 기울이게 돼.

이렇게 중요한 정보에 더 집중하고, 덜 중요한 정보는 조금 덜 듣는 것, 이게 바로 어텐션이야. 컴퓨터가 문장을 이해할 때도 비슷한 방식으로, 어떤 단어가 다른 단어와 더 관계가 있는지를 알아보는 거야.

예를 들어, "나의 강아지는 공원에서 뛰고 있다"라는 문장을 생각해보면, 컴퓨터는 "강아지"와 "뛰고 있다"라는 단어 사이의 관계가 중요하다는 걸 이해할 수 있어. 그래서 어텐션은 컴퓨터가 어떤 단어에 더 집중해야 할지를 결정해주는 도와주는 역할을 해.

이해가 되었나요? 더 궁금한 점이 있다면 언제든지 물어봐!
        """},
        {'role': 'user', 'content': '이해가 되는 것 같아. 트랜스포머 내용을 요약해서 하나의 markdown 문서로 정리해줘.'}
    ]
)
print(response.choices[0].message.content)

물론이야! 트랜스포머 모델에 대한 내용을 요약한 Markdown 문서를 아래에 작성해볼게.

```markdown
# Transformer 모델 개요

## 1. 소개
Transformer는 2017년에 제안된 자연어 처리 모델로, 자기 주의(Self-Attention) 메커니즘을 활용하여 입력 시퀀스의 각 단어 간의 관계를 효과적으로 학습한다.

## 2. 구조

### 2.1 인코더 (Encoder)
- 입력 데이터를 처리하고 고차원 벡터 표현으로 변환.
- 여러 개의 레이어로 구성되며, 각 레이어는 다음으로 구성된다:
  - 자기 주의 메커니즘
  - 피드포워드 신경망

### 2.2 디코더 (Decoder)
- 인코더의 출력을 받아 최종 출력을 생성.
- 여러 개의 레이어로 구성되며, 각 레이어는 다음으로 구성된다:
  - 자기 주의 메커니즘
  - 인코더의 출력을 참조하는 메커니즘

### 2.3 자기 주의 메커니즘 (Self-Attention)
- 입력 시퀀스 내의 단어들 간의 관계를 파악.
- 각 단어는 다른 단어들과의 관계를 고려하여 표현을 업데이트한다.

### 2.4 포지셔널 인코딩 (Positional Encoding)
- 시퀀스 내에서 각 단어의 위치 정보를 제공.
- Transformer는 순서에 대한 정보를 내재하지 않기 때문에 필요하다.

## 3. 장점
- 병렬 처리 가능: 순차적으로 처리할 필요가 없으므로 계산 속도가 빠르다.
- 높은 성능: 자연어 처리 작업에서 뛰어난 성능을 발휘한다.

## 4. 활용 예시
- BERT, GPT, T5와 같은 다양한 NLP 모델들이 Transformer를 기반으로 하고 있다.

## 5. 마무리
Transformer 모델은 현대 자연어 처리에서 중요한 기술로 자리 잡고 있으며, 자기 주의 메커니즘을 통해 문맥을 이해하는 데 있어 상당한 기여를 하고 있다.
```

이 문서를 참고해서 Transformer 모델에 대한 이해를 돕는 데 활용해보세요! 추가적으로 궁금한 점이 있다면 언제든지 질문해 

## 반복처리

In [15]:
# 대화내역을 로깅
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕, 나 윤두준이야~
Assistant: 안녕하세요, 윤두준님! 만나서 정말 반가워요 😊 오늘은 어떤 이야기를 나눠볼까요? 궁금한 것이나 도움이 필요한 게 있다면 언제든 말씀해 주세요!
User: 이삭토스트 메뉴 알려줘
Assistant: 네, 윤두준님! 이삭토스트는 다양한 토스트 메뉴로 유명하죠. 이삭토스트의 대표적인 메뉴와 인기 메뉴를 알려드릴게요.  
(매장마다 약간씩 다를 수 있으니 참고해 주세요.)

**이삭토스트 인기 메뉴 예시**

1. **햄치즈 토스트**  
  - 햄, 치즈, 계란, 채소, 특제 소스

2. **베이컨 베스트**  
  - 베이컨, 해시브라운, 계란, 치즈, 채소, 특제 소스

3. **불고기 MVP**  
  - 불고기, 치즈, 계란, 채소, 특제 소스

4. **클럽 샌드위치**  
  - 햄, 터키, 계란, 치즈, 채소, 특제 소스

5. **더블치즈 포테이토**  
  - 치즈 두 장, 해시브라운, 계란, 채소, 특제 소스

6. **핫치킨 마요**  
  - 매콤한 치킨, 계란, 채소, 치즈, 마요네즈 소스

7. **치즈 스테이크**  
  - 소고기, 피망, 양파, 치즈, 계란, 채소, 특제 소스

8. **스위트 베이컨**  
  - 베이컨, 계란, 치즈, 채소, 달콤한 소스

그 외에도 **스팸 토스트**, **참치마요 토스트**, **더블햄 토스트** 등등 다양한 메뉴가 있어요.

음료 메뉴로는 커피, 쥬스, 에이드 등도 함께 판매하고 있습니다.

궁금한 특정 메뉴나 추천받고 싶은 조합이 있으면 말씀해 주세요!
User: 아냐 맘스터치가 좋겠어
Assistant: 좋은 선택이에요, 윤두준님! 맘스터치는 버거와 치킨이 정말 인기 많죠.  
대표적인 맘스터치 메뉴들을 알려드릴게요.

---

### 🍔 버거 메뉴  
- **싸이버거** : 맘스터치의 시그니처! 두툼한 치킨패티와 신선한 채소, 소스 조화가 일품이에요.
- **딥치즈싸이버거** : 싸이버거에 고소한

## stream

In [19]:
stream = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role': 'user', 'content': '지금 stream 테스트 할거야. 아주 긴 응답 메세지를 보내줘.'}],
    stream=True
)
for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end='')

물론입니다! 아래는 긴 응답 메시지입니다. 필요한 정보나 주제가 있다면 말씀해 주세요.

---

안녕하세요! 오늘은 여러 가지 주제에 대해 긴 메시지를 작성해 보겠습니다. 

**1. 자연의 아름다움**  
자연은 우리가 사는 세상에서 가장 경이로운 것 중 하나입니다. 아름다운 풍경, 신비로운 동식물, 그리고 사계절의 변화는 모두 인류에게 큰 영감을 줍니다. 봄에는 꽃들이 피어나고, 여름에는 푸른 나무와 해변의 따뜻한 햇살을 즐길 수 있습니다. 가을에는 단풍이 아름답게 물들고, 겨울에는 눈에 덮인 풍경이 새로운 감동을 선사합니다. 이러한 자연의 변화를 통해 우리는 삶의 소중함과 경이로움을 느끼게 됩니다.

**2. 과학의 발전**  
최근 몇 년 동안 과학 기술은 급속도로 발전해왔습니다. 인공지능(AI), 유전자 편집(CRISPR), 우주 탐사 등이 대표적입니다. AI는 이미 우리의 일상 생활에 깊숙이 들어와 있으며, 자율주행차, 스마트 홈, 인공지능 비서 등 다양한 형태로 우리의 삶을 변화시키고 있습니다. 유전자 편집 기술은 질병의 치료와 예방에서 혁신적인 가능성을 보여줍니다. 이처럼 과학의 발전은 우리의 삶의 질을 향상시키고, 새로운 지식을 통해 인류의 미래를 밝히는 데 기여하고 있습니다.

**3. 문화와 예술**  
문화와 예술은 인류의 역사에서 중요한 역할을 해왔습니다. 음악, 미술, 문학 등 다양한 형태의 예술은 사람들의 감정을 표현하고 소통하는 방법 중 하나입니다. 전통 문화를 계승하면서도 현대적인 요소를 결합하여 새로운 예술 장르가 탄생하고 있습니다. 예술을 통해 우리는 서로 다른 배경과 경험을 가진 사람들과의 공감대를 형성할 수 있습니다. 또한, 예술은 사회 비판의 도구로 사용되기도 하며, 각 시대의 문제를 반영하는 거울 역할을 합니다.

**4. 사회의 변화**  
21세기는 빠르게 변화하는 사회로 특징지어집니다. 디지털화, 글로벌화, 그리고 정보의 폭발적인 증가로 인해 우리의 삶은 점점 더 복잡해지고 있습니다. 직장 환경, 교육 시스템, 

In [20]:
# stream 버전
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    stream = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1,
        stream=True
    )
    print(f'Assistant: ', end='')
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='', flush=True) # 내부 buffer 사용하지 않음
    print()

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요...
User: 긴 응답
Assistant: 물론입니다! 긴 응답을 원하시는군요. 어떤 주제에 대해 긴 설명이나 답변을 듣고 싶으신가요? 특정한 주제가 있으시다면 알려주시면, 그에 맞춰 최대한 자세하고 길게 설명드릴 수 있습니다.

예를 들어, "인공지능의 역사", "효과적인 공부 방법", "기후 변화의 원인과 영향" 등 관심 있는 주제를 말씀해주시면 됩니다.

혹시 주제를 정하지 않았다면, '인공지능의 원리와 미래'에 대해 길게 설명드릴게요.

---

**인공지능의 원리와 미래**

인공지능(AI)은 인간의 학습, 추론, 문제 해결 능력을 컴퓨터 시스템으로 구현하는 기술입니다. AI는 오랜 시간에 걸쳐 발전해왔고, 초기에는 단순한 규칙 기반 시스템에서 출발하여 최근에는 딥러닝(Deep Learning)과 같은 첨단 기술을 바탕으로 혁신적인 변화를 이끌고 있습니다.

**기본 원리**

인공지능의 기본은 데이터를 바탕으로 패턴을 인식하고, 그 패턴을 통해 의사결정을 내리는 데 있습니다. 초창기의 인공지능은 주로 사람이 입력한 규칙에 따라 동작했지만, 최근의 AI는 방대한 데이터를 학습하여 스스로 규칙을 발견하는 '기계학습(Machine Learning)'을 기반으로 합니다. 딥러닝은 기계학습의 한 분야로, 사람이 뇌에서 정보를 처리하는 방식과 유사한 인공 신경망(Artificial Neural Networks)을 활용합니다.

**AI의 적용 분야**

AI는 오늘날 다양한 곳에서 활용되고 있습니다.

1. **음성 인식과 자연어 처리**  
   AI는 사람의 음성을 인식하여 텍스트로 바꾸거나, 자연어 처리를 활용해 대화를 이해하고 대응합니다. 스마트폰의 음성 비서, 챗봇, 번역 서비스 등이 대표적인 예입니다.

2. **이미지 인식**  
   의료 영상 진단, 자율주행 자동차의 장애물 인식 등에서 AI는 이미지를 분석하고 사물을 구분할 수 있습니다.

3. **추천 시스템**  
   넷플릭스, 유튜브, 쇼핑 사이트 등

# Token counting

비용 = ((입력 토큰 수 * 단가) + (출력 토큰 수 * 단가)) * 월 서비스 호출 수

In [21]:
!pip install tiktoken

In [27]:
# 각 모델에 따른 토크나이저(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o-mini') # gpt-4o, gpt-4o-mini
# gpt41 = tiktoken.encoding_for_model('gpt-4.1')
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35)
print(gpt4o)
print(gpt41)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>
<Encoding 'cl100k_base'>


In [26]:
# 토큰 수 세기
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다.')
encoded_gpt4o = gpt4o.encode('아버지가 방에 들어가신다.')

print(len(encoded_gpt35))
print(len(encoded_gpt4o))

13
10


## 토큰 비용 계산하기

In [28]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_gpt4o = gpt4o.encode(text)
encoded_text_gpt41 = gpt41.encode(text)

print("gpt-4o 토큰 수: ", len(encoded_text_gpt4o))
print("gpt-4.1 토큰 수: ", len(encoded_text_gpt41))

gpt-4o 토큰 수:  703
gpt-4.1 토큰 수:  1215


In [31]:
response_gpt4o = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

output_gpt4o = response_gpt4o.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print("gpt-4o 응답: ", output_gpt4o)
print("gpt-4.1 응답: ", output_gpt41)

print('\n', 'gpt-4o 토큰수: ', len(gpt4o.encode(output_gpt4o)))
print('gpt-4.1 토큰수: ', len(gpt41.encode(output_gpt41)))

gpt-4o 응답:  더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성하는 입장을 시사하자, 탄핵소추안 표결 시점을 앞당기는 방안을 검토하고 있습니다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라며 신중한 태도를 보이고 있습니다. 민주당은 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 표결 시점을 오후 7시에서 5시로 앞당기는 방안도 논의 중입니다. 민주당은 국민의힘의 지연 전략을 고려해 신속한 대응을 준비하고 있습니다.
gpt-4.1 응답:  ### 기사 핵심 요약

**1. 민주당, 윤석열 대통령 탄핵소추안 표결 시점 앞당기나**
- 더불어민주당이 7일로 예정된 윤석열 대통령 탄핵소추안 표결을 앞당기는 방안을 검토 중임.
- 한동훈 국민의힘 대표가 윤 대통령의 직무 정지 필요성을 언급하며 사실상 탄핵에 찬성하는 듯한 입장을 시사한 것이 계기.

**2. 이재명 대표, 신중론 강조**
- 이재명 민주당 대표는 한동훈 대표의 입장 변화가 확실한지 먼저 파악해야 한다며 신중한 태도를 보임.
- 한동훈 대표와의 회동 요청도 했으나 아직 확정되지 않음.

**3. 민주당, 긴급 의원총회 소집**
- 한동훈 대표의 발언 이후 민주당은 긴급 의원총회를 열어 당내 의견을 수렴 중.
- 표결 시점 변경(예정된 오후 7시 → 오후 5시)도 논의되고 있음.

**4. 긴장 고조 및 우려**
- 민주당은 최근 정치인 체포 시도 등으로 상황이 긴박하다고 판단, 비상 대응에 나섬.
- 이재명 대표는 ‘2차 계엄’ 가능성 등 추가적인 돌발 상황을 우려.

**5. 표결 시점 변경은 아직 미정**
- 본회의 일정 변경은 아직 확정되지 않았으며, 국민의힘의 지연 전략 가능성도 고려 중.

---

**핵심 정리:**  
한동훈 대표의 입장 변화로 민주당이 윤 대통령 탄핵소추안 표결을 앞당길지 고심 중이나, 이재명 대표는 신중한 접근을 강조. 민주당은 긴급 의원총회를 통해 대응 방안을 논의하고 있으며, 표결 시점 변경 가능성도 열어두고 

In [33]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
    if model == 'gpt-4.1':
        encoding = tiktoken.get_encoding('cl100k_base')
    else:
        encoding = tiktoken.encoding_for_model(model)
    encoded = encoding.encode(text)
    return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)

    # 모델별 단가 가져오기
    price = PRICING[model]

    # 비용 계산
    input_cost = (input_tokens / 1_000_000) * price['input']
    output_cost = (output_tokens / 1_000_000) * price['output']

    total_cost = (input_cost + output_cost) * num_service_call
    return total_cost

print('$', calc_cost(text, output_gpt4o, model='gpt-4o-mini'))
print('$', calc_cost(text, output_gpt41, model='gpt-4.1'))


$ 196.64999999999998
$ 8142.0
